---
title: "Cómo corregir el optimismo de tu modelo estadístico"
description: "Bootstrap aplicado a la corrección del optimismo de un modelo estadístico cuando tienes pocos datos."
description-meta: "Bootstrap aplicado a la corrección del optimismo de un modelo estadístico cuando tienes pocos datos."
author: "Leonardo Hansa"
date: "2025-04-17"
categories: [exploraciones]
execute: 
  echo: true
  eval: true
  message: false
  warning: false
freeze: true
---


## Comentarios iniciales

Si entrenas un modelo en un conjunto de datos que no es muy grande, la métrica de ajuste que reportes no deberías calcularla sobre los datos de entrenamiento. Esto es porque el modelo ya conoce esos datos y se ha entrenado con ellos, intentando optimizar esa métrica de ajuste. 

Lo típico es reservar un conjunto de validación, unos datos que el modelo no conoce, por lo que la métrica de ajuste no tendrá ese sesgo. 

A ese sesgo lo llamamos _optimismo._

Lo malo, según Frank Harrell, es que, si tu conjunto de datos es pequeño, esa validación no será suficiente estable.

_Bootstrap_ es una solución. 


In [ ]:
#| label: libs
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.utils import resample
import matplotlib.pyplot as plt

## Bootstrap para corregir el optimismo

La idea es entrenar el modelo en varias muestras bootstrap y calcular la métrica de ajuste en cada par muestra bootstrap, muestra original, y luego la diferencia. Así, tendrás una lista de diferencias de longitud el número de muestras. 

Luego calculas la media. 

Finalmente, entrenas el modelo en la muestra original completa y le aplicas esa diferencia media. 

La métrica de ajuste final será la métrica de ajuste en entrenamiento menos la diferencia, es decir, con el optimismo corregido.

### Algunos datos


In [ ]:
#| label: load
data = fetch_openml(name="boston", version=1, as_frame=True)
X = data.data
y = data.target

X_np = X.to_numpy()
y_np = y.to_numpy()

### Entrenamiento en muestras bootstrap


In [ ]:
#| label: set-up-boots
n_bootstraps = 200

Lo siguiente creo que quedaría más claro con un bucle `for`, pero en teoría no están recomendados. Así que creo una función y la llamo en una _list comprehension_. 


In [ ]:
#| label: compute-optimism
def compute_optimism():
    X_boot, y_boot = resample(X_np, y_np)

    model = LinearRegression()
    model.fit(X_boot, y_boot)

    y_pred_boot = model.predict(X_boot)
    r2_boot = r2_score(y_boot, y_pred_boot)

    y_pred_orig = model.predict(X_np)
    r2_orig = r2_score(y_np, y_pred_orig)

    return(r2_boot - r2_orig)

Y ahora calculo todo.


In [ ]:
#| label: list-optimism
optimism_estimates = [compute_optimism() for _ in range(n_bootstraps)]

### Cálculo del optimismo

El optimismo medio es lo que necesito para el próximo paso. Así que calculo la media de la lista que acabo de generar.


In [ ]:
#| label: optimism
mean_optimism = np.mean(optimism_estimates)
mean_optimism

Por curiosidad, así se distribuye el optimismo.


In [ ]:
#| label: histogram
plt.figure(figsize=(8, 5))
plt.hist(optimism_estimates, bins=20, color="#800080", edgecolor="black", alpha=0.75)

plt.title(
    "Distribución del optimismo del modelo (bootstrap)", fontsize=14, fontweight="bold"
)
plt.xlabel("Optimismo estimado", fontsize=12)
plt.ylabel("Frecuencia", fontsize=12)

plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.3)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

### Entrenamiento final

Todos los entrenamientos anteriores estaban hechos sobre muestras _bootrstrap_, es decir, no estaban sobre el entrenamiento


In [ ]:
#| label: final-train
final_model = LinearRegression()
final_model.fit(X_np, y_np)
final_r2 = r2_score(y_np, final_model.predict(X_np))

In [ ]:
#| label: optimism-correct
corrected_r2 = final_r2 - mean_optimism

print(f"R² original: {final_r2:.4f}")
print(f"Optimismo medio: {mean_optimism:.4f}")
print(f"R² corregido: {corrected_r2:.4f}")